<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>



<br>

# <font color="#76b900">**Notebook 4:** Running State Chains</font>

<br>

在前一個 notebook 中，我們介紹了一些關於可運行物件(Runnable)的關鍵 LangChain 表達式語言 (LCEL) 材料。到現在為止，您應該理解對內部和外部分析推理(Reasoning)以及如何開發促進這些功能的管線(Pipeline)！在這個 notebook 中，我們將邁向更高級的作法(Paradigm)，這些作法(Paradigm)將允許我們流程協調管理(Orchestration)更複雜的對話管理策略，並開始執行長篇的(long-form)文件分析推理(Reasoning)。

**學習目標：**


-   學習如何利用可運行物件(Runnable)來流程協調管理(Orchestration)有趣的 LLM 系統。
-   了解如何使用 Running State Chains 進行對話管理和疊代(Iterate)決策制定。

**值得思考的問題：**


-   是否會有使用 Running State Chain 的單一模組變種(variant)的用途，該變種(variant)不會持續查詢環境以獲取輸入(Intake)？
-   您可能會注意到 JSON 預測實際上運作得相當好。根據問題和 JSON 格式複雜性，它可能不會總是運作得這麼好。您預期在這方面會遇到什麼樣的問題？
-   您能想到什麼樣的方法可以完全交換提示(Prompt)作為 Running State Chain 的一部分？
<br>

### **環境設置：**





In [1]:
## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio

# import os
# os.environ["NVIDIA_API_KEY"] = "nvapi-..."

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [7]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
ChatNVIDIA.get_available_models()

/usr/local/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/_common.py:237: UserWarning: Default model is set as: 01-ai/yi-large. 
Set model using model parameter. 
To get available models use available_models property.
  warnings.warn(


[Model(id='01-ai/yi-large', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-yi-large'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='abacusai/dracarys-llama-3.1-70b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='ai21labs/jamba-1.5-large-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='ai21labs/jamba-1.5-mini-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='aisingapore/sea-lion-7b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=['ai-sea-lion-7b-instruct'], supports_tools=False, supports_structured_output=False, base_model=None),
 Model(id='baichuan-inc/baichu

In [8]:
## Useful utility method for printing intermediate states
from langchain_core.runnables import RunnableLambda
from functools import partial

def RPrint(preface="State: "):
    def print_and_return(x, preface=""):
        print(f"{preface}{x}")
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def PPrint(preface="State: "):
    def print_and_return(x, preface=""):
        pprint(preface, x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))



<br>

## **第一部分：** 保持變數流動


在之前的範例中，我們能夠透過 **創建(creating)** 、 **變更(mutating)** 和 **消費(consuming)** 狀態在我們的獨立鏈(Chain)中實作有趣的邏輯。這些狀態以具有描述性鍵值(descriptive keys)和有用值的字典形式傳遞，這些值將被用來為後續程式提供它們運作所需的資訊！

**回想上一個 notebook 中的零樣本(Zero-Shot)分類範例：**




In [17]:
%%time
## ^^ This notebook is timed, which will print out how long it all took

from langchain_core.runnables import RunnableLambda
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Union
from operator import itemgetter

## Zero-shot classification prompt and chain w/ explicit few-shot prompting
sys_msg = (
    "Choose the most likely topic classification given the sentence as context."
    " Only one word, no explanation.\n[Options : {options}]"
)

zsc_prompt = ChatPromptTemplate.from_template(
    f"{sys_msg}\n\n"
    "[[The sea is awesome]][/INST]boat</s><s>[INST]"
    "[[{input}]]"
)

## Define your simple instruct_model
# instruct_chat = ChatNVIDIA(model="mistralai/mistral-7b-instruct-v0.2")
instruct_chat = ChatNVIDIA(model="google/gemma-2-27b-it")
instruct_llm = instruct_chat | StrOutputParser()
one_word_llm = instruct_chat.bind(stop=[" ", "\n"]) | StrOutputParser()

zsc_chain = zsc_prompt | one_word_llm

## Function that just prints out the first word of the output. With early stopping bind
def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options}).split()[0]

print("-" * 80)
print(zsc_call("Should I take the next exit, or keep going to the next one?"))

print("-" * 80)
print(zsc_call("I get seasick, so I think I'll pass on the trip"))

print("-" * 80)
print(zsc_call("I'm scared of heights, so flying probably isn't for me"))

--------------------------------------------------------------------------------
car
--------------------------------------------------------------------------------
boat
--------------------------------------------------------------------------------
airplane
CPU times: user 28.1 ms, sys: 530 μs, total: 28.7 ms
Wall time: 1.43 s


<br>


這個鏈(Chain)做出了幾個設計決策，使其非常容易使用，其中關鍵的包括以下內容：

**我們希望它像函式(function)一樣運作，所以我們希望它做的就是生成輸出並返回它。**

這使得鏈(Chain)在作為更大鏈(Chain)系統中的模組時極其自然。例如，以下鏈(Chain)將接受一個字串，提取最可能的主題，然後基於該主題生成一個新句子：

In [18]:
%%time
## ^^ This notebook is timed, which will print out how long it all took
gen_prompt = ChatPromptTemplate.from_template(
    "Make a new sentence about the the following topic: {topic}. Be creative!"
)

gen_chain = gen_prompt | instruct_llm

input_msg = "I get seasick, so I think I'll pass on the trip"
options = ["car", "boat", "airplane", "bike"]

chain = (
    ## -> {"input", "options"}
    {'topic' : zsc_chain}
    | PPrint()
    ## -> {**, "topic"}
    | gen_chain
    ## -> string
)

chain.invoke({"input" : input_msg, "options" : options})

State: 
{'topic': 'boat'}

CPU times: user 19.4 ms, sys: 2.36 ms, total: 21.8 ms
Wall time: 1.51 s


'The little sailboat, christened "Wanderlust," danced with the waves, its sails whispering tales of faraway shores.'

<br>


然而，當您想要保持資訊流動時，這有點問題，因為我們在生成回應時失去了主題和輸入(Intake)變數。如果我們想要對輸出和輸入(Intake)都做些什麼，我們需要一種方法來確保兩個變數都通過。

幸運的是，我們可以使用映射(mapping) Runnable（即從字典解釋或使用手動 `RunnableMap`）透過將我們鏈(Chain)的輸出分配給單一鍵並讓其他鍵按需要傳播來傳遞兩個變數。或者，我們也可以使用 `RunnableAssign` 將狀態消費鏈(Chain)的輸出與輸入(Intake)字典預設合併。

使用這種技術，我們可以透過我們的鏈(Chain)系統傳播任何我們想要的東西：

In [21]:
%%time
## ^^ This notebook is timed, which will print out how long it all took

from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.schema.runnable.passthrough import RunnableAssign
from functools import partial

big_chain = (
    PPrint()
    ## Manual mapping. Can be useful sometimes and inside branch chains
    # | {'input' : lambda d: d.get('input'), 'topic' : zsc_chain}
    |    RunnableAssign({"topic": zsc_chain})
    | PPrint()
    ## RunnableAssign passing. Better for running state chains by default
    | RunnableAssign({'generation' : gen_chain})
    | PPrint()
    ## Using the input and generation together
    | RunnableAssign({'combination' : (
        ChatPromptTemplate.from_template(
            "Consider the following passages:"
            "\nP1: {input}"
            "\nP2: {generation}"
            "\n\nCombine the ideas from both sentences into one simple one."
        )
        | instruct_llm
    )})
)

output = big_chain.invoke({
    "input" : "I get seasick, so I think I'll pass on the trip",
    "options" : ["car", "boat", "airplane", "bike", "unknown"],
    'test':'tt'
})
pprint("Final Output: ", output)

State: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'options': ['car', 'boat', 'airplane', 'bike', 'unknown'],
    'test': 'tt'
}

State: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'options': ['car', 'boat', 'airplane', 'bike', 'unknown'],
    'test': 'tt',
    'topic': 'boat'
}

State: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'options': ['car', 'boat', 'airplane', 'bike', 'unknown'],
    'test': 'tt',
    'topic': 'boat',
    'generation': 'The little sailboat, christened "Wanderlust," danced on the waves, its sails whispering tales of
faraway shores.'
}

Final Output: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'options': ['car', 'boat', 'airplane', 'bike', 'unknown'],
    'test': 'tt',
    'topic': 'boat',
    'generation': 'The little sailboat, christened "Wanderlust," danced on the waves, its sails whispering tales of
faraway shores.',
    'combination': 'Although the sailboat "Wanderlust" looked inviting, I\'ll have to skip the trip because I get 
seasick.'
}

CPU times: user 49.8 ms, sys: 13.6 ms, total: 63.4 ms
Wall time: 2.72 s


In [22]:
%%time
## ^^ This notebook is timed, which will print out how long it all took

from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.schema.runnable.passthrough import RunnableAssign
from functools import partial

big_chain = (
    PPrint()
    ## Manual mapping. Can be useful sometimes and inside branch chains
    | {'input' : lambda d: d.get('input'), 'topic' : zsc_chain}
    | PPrint()
    ## RunnableAssign passing. Better for running state chains by default
    | RunnableAssign({'generation' : gen_chain})
    | PPrint()
    ## Using the input and generation together
    | RunnableAssign({'combination' : (
        ChatPromptTemplate.from_template(
            "Consider the following passages:"
            "\nP1: {input}"
            "\nP2: {generation}"
            "\n\nCombine the ideas from both sentences into one simple one."
        )
        | instruct_llm
    )})
)

output = big_chain.invoke({
    "input" : "I get seasick, so I think I'll pass on the trip",
    "options" : ["car", "boat", "airplane", "bike", "unknown"],
    'test':'tt'
})
pprint("Final Output: ", output)

State: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'options': ['car', 'boat', 'airplane', 'bike', 'unknown'],
    'test': 'tt'
}

State: 
{'input': "I get seasick, so I think I'll pass on the trip", 'topic': 'boat'}

State: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'topic': 'boat',
    'generation': 'The little sailboat, christened "Wanderlust," danced with the waves, its sails whispering tales 
of faraway shores.'
}

Final Output: 
{
    'input': "I get seasick, so I think I'll pass on the trip",
    'topic': 'boat',
    'generation': 'The little sailboat, christened "Wanderlust," danced with the waves, its sails whispering tales 
of faraway shores.',
    'combination': 'Although the sailboat "Wanderlust" looked inviting, I\'ll have to skip the trip because I get 
seasick.'
}

CPU times: user 57 ms, sys: 2.18 ms, total: 59.2 ms
Wall time: 2.64 s




<br>

## **第二部分：** Running State Chain


上面的範例只是一個玩具範例，如果有什麼的話，它展示了將許多 LLM 呼叫鏈接在一起進行內部幕後分析推理(Reasoning)的缺點。然而，保持資訊透過鏈(Chain)流動的能力對於製作能夠累積有用狀態資訊或以多次通過能力運作的複雜鏈(Chain)是無價的。

具體來說，一個非常簡單但有效的鏈(Chain)是 **Running State Chain**，它強制執行以下屬性：

-   **「運行狀態」** 是一個包含系統關心的所有變數的字典。

-   **「分支(Branch)」** 是一個可以拉入running state並可以將反生成(degenerate)為回應的鏈(Chain)。

-   **分支(Branch)** 只能在 **RunnableAssign** 範圍內運行，分支(Branch)的輸入(Intake)應該來自 **運行狀態**。

> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/running_state_chain.png" width=1000px/>
<!-- > <img src="https://drive.google.com/uc?export=view&id=1Oo7AauYGj4dxepNReRG2JezmvQLyqXsN" width=1000px/> -->



您可以將 Running State Chain 抽象化(abstractions)視為具有狀態變數(state variables)（或屬性）和函式(function)（或方法）的 Pythonic 類別的函式(function)變體。

-   鏈(Chain)就像包裝所有功能的抽象類別。
-   運行狀態(running state)就像屬性（應該始終可存取）。
-   分支(Branch)就像類別方法（可以挑選使用哪些屬性）。
-   `.invoke` 或類似過程就像按順序運行分支(Branch)的 `__call__` 方法。

**透過在您的鏈(Chain)中強制執行這個作法(Paradigm)：**

-   您可以保持狀態變數透過您的鏈(Chain)傳播，允許您的內部存取任何必要的內容並累積狀態值供以後使用。
-   您也可以將鏈(Chain)的輸出作為輸入(Intake)傳回，允許「while 迴圈」風格的鏈(Chain)持續更新和建構您的運行狀態。

這個 notebook 的其餘部分將包括兩個練習，為兩個額外的使用案例：**知識庫(Knowledge Bases)** 和 **資料庫查詢聊天機器人(Database-Querying Chatbots)** 強化 Running State Chain 的概念。


<br>

## **第三部分：** 使用 Running State Chain 實作知識庫


在理解 Running State Chain 的基本結構和原則後，我們可以探索如何將這種方法擴展到管理更複雜的任務，特別是在創建透過互動演化的動態系統方面。本節將專注於使用 **json 啟用的空缺填充(slot filling)** 實作**知識庫** 的累積：

-   **知識庫：** 與我們的 LLM 保持追蹤相關的資訊存儲。
-   **基於JSON的空缺填充(slot filling)：** 要求指令調整模型輸出 json 風格格式（可以包括字典）與空缺選擇的技術，依賴 LLM 用有用和相關的資訊填充這些空缺。

<br>

#### **定義我們的知識庫**


要建構一個回應式(responsive)和智慧的系統，我們需要一種不僅處理輸入(Intake)而且透過對話流程保留和更新基本資訊的方法。這就是 LangChain 和 Pydantic 的結合變得關鍵的地方。[**Pydantic**](https://docs.pydantic.dev/latest/)，一個常見(popular)的 Python 驗證函式庫(library)，在結構化和驗證資料模型方面發揮重要作用。作為其功能之一，Pydantic 提供結構化的「模型(model)」類別，用簡化的語法和深度自訂選項的深層兔子洞(rabbitholes)驗證物件（資料、類別、它們自己等）。這個框架在整個 LangChain 中使用，並作為涉及資料強制的使用案例的必要組件出現。

「模型(model)」非常擅長的一件事是定義具有預期參數和一些特殊驗證方式的類別！在這門課程中，我們不會過多關注驗證腳本(scripts)，但感興趣的人可以從查看 [**Pydantic Validator 指南**](https://docs.pydantic.dev/1.10/usage/validators/) 開始（儘管主題確實很快變得相當深入）。對於我們的目的，我們可以構建一個 `BaseModel` 類別並定義一些 `Field` 變數來創建結構化的 **知識庫**，如下所示：

In [23]:
from pydantic import BaseModel, Field
from typing import Dict, Union, Optional

instruct_chat = ChatNVIDIA(model="mistralai/mistral-7b-instruct-v0.2")

class KnowledgeBase(BaseModel):
    ## Fields of the BaseModel, which will be validated/assigned when the knowledge base is constructed
    topic: str = Field('general', description="Current conversation topic")
    user_preferences: Dict[str, Union[str, int]] = Field({}, description="User preferences and choices")
    session_notes: list = Field([], description="Notes on the ongoing session")
    unresolved_queries: list = Field([], description="Unresolved user queries")
    action_items: list = Field([], description="Actionable items identified during the conversation")

print(repr(KnowledgeBase(topic = "Travel")))

KnowledgeBase(topic='Travel', user_preferences={}, session_notes=[], unresolved_queries=[], action_items=[])


<br>


這種方法的真正優勢在於 LangChain 提供的額外以 LLM 為中心的功能，我們可以為我們的使用案例整合這些功能。其中一個這樣的功能是 `PydanticOutputParser`，它透過自動格式指令生成等功能增強 Pydantic 物件。

In [24]:
from langchain.output_parsers import PydanticOutputParser

instruct_string = PydanticOutputParser(pydantic_object=KnowledgeBase).get_format_instructions()
pprint(instruct_string)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": 
"array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": 
["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"topic": {"default": "general", "description": "Current conversation topic", "title": "Topic", 
"type": "string"}, "user_preferences": {"additionalProperties": {"anyOf": [{"type": "string"}, {"type": 
"integer"}]}, "default": {}, "description": "User preferences and choices", "title": "User Preferences", "type": 
"object"}, "session_notes": {"default": [], "description": "Notes on the ongoing session", "items": {}, "title": 
"Session Notes", "type": "array"}, "unresolved_queries": {"default": [], "description": "Unresolved user queries", 
"items": {}, "title": "Unresolved Queries", "type": "array"}, "action_items": {"default": [], "description": 
"Actionable items identified during the conversation", "items": {}, "title": "Action Items", "type": "array"}}}
```



這個功能生成創建知識庫有效輸入(Intake)的指令，這反過來透過提供所需輸出格式的具體一次性樣本範例(one-shot example)來幫助 LLM。

<br>

#### **可運行物件(Runnable)提取模組**


知道我們有這個可以用來生成良好 LLM 指令的 Pydantic 物件，我們可以製作一個包裝我們 Pydantic 類別功能並簡化知識庫的提示(Prompt)、生成和更新Runnable：

In [25]:
################################################################################
## Definition of RExtract
def RExtract(pydantic_class, llm, prompt):
    '''
    Runnable Extraction module
    Returns a knowledge dictionary populated by slot-filling extraction
    '''
    parser = PydanticOutputParser(pydantic_object=pydantic_class)
    instruct_merge = RunnableAssign({'format_instructions' : lambda x: parser.get_format_instructions()})
    def preparse(string):
        if '{' not in string: string = '{' + string
        if '}' not in string: string = string + '}'
        string = (string
            .replace("\\_", "_")
            .replace("\n", " ")
            .replace("\]", "]")
            .replace("\[", "[")
        )
        # print(string)  ## Good for diagnostics
        return string
    return instruct_merge | prompt | llm | preparse | parser

################################################################################
## Practical Use of RExtract

parser_prompt = ChatPromptTemplate.from_template(
    "Update the knowledge base: {format_instructions}. Only use information from the input."
    "\n\nNEW MESSAGE: {input}"
)

extractor = RExtract(KnowledgeBase, instruct_llm, parser_prompt)

knowledge = extractor.invoke({'input' : "I love flowers so much! The orchids are amazing! Can you buy me some?"})
pprint(knowledge)

KnowledgeBase(
    topic='flowers',
    user_preferences={},
    session_notes=[],
    unresolved_queries=['Can you buy me some?'],
    action_items=[]
)

<br>




請記住，由於 LLM 預測的模糊性質，這個過程可能會失敗，特別是對於未針對指令跟隨最佳化的模型！對於這個過程，重要的是要有一個強大的指令跟隨 LLM，具有額外的檢查和優雅的失敗例程。

<br>

#### **動態知識庫更新**


最後，我們可以創建一個在整個對話中持續更新知識庫的系統。這是透過將知識庫的當前狀態與新的使用者輸入(Intake)一起回饋到系統中進行持續更新來完成的。

以下是一個範例系統，展示了填充細節的制定能力以及假設填充效能(filling performance )將與一般回應效能一樣好的限制：

In [26]:
class KnowledgeBase(BaseModel):
    firstname: str = Field('unknown', description="Chatting user's first name, unknown if unknown")
    lastname: str = Field('unknown', description="Chatting user's last name, unknown if unknown")
    location: str = Field('unknown', description="Where the user is located")
    summary: str = Field('unknown', description="Running summary of conversation. Update this with new input")
    response: str = Field('unknown', description="An ideal response to the user based on their new message")


parser_prompt = ChatPromptTemplate.from_template(
    "You are chatting with a user. The user just responded ('input'). Please update the knowledge base."
    " Record your response in the 'response' tag to continue the conversation."
    " Do not hallucinate any details, and make sure the knowledge base is not redundant."
    " Update the entries frequently to adapt to the conversation flow."
    "\n{format_instructions}"
    "\n\nOLD KNOWLEDGE BASE: {know_base}"
    "\n\nNEW MESSAGE: {input}"
    "\n\nNEW KNOWLEDGE BASE:"
)

## Switch to a more powerful base model
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1") | StrOutputParser()

extractor = RExtract(KnowledgeBase, instruct_llm, parser_prompt)
info_update = RunnableAssign({'know_base' : extractor})

## Initialize the knowledge base and see what you get
state = {'know_base' : KnowledgeBase()}
state['input'] = "My name is Carmen Sandiego! Guess where I am! Hint: It's somewhere in the United States."
state = info_update.invoke(state)
pprint(state)

{
    'know_base': KnowledgeBase(
        firstname='Carmen',
        lastname='Sandiego',
        location='unknown',
        summary='The user introduced themselves as Carmen Sandiego and asked for a guess on their location within 
the United States, providing a hint.',
        response="Welcome, Carmen Sandiego! I'm excited to try and guess your location. Since you mentioned it's 
somewhere in the United States, I'll start thinking of places within its borders."
    ),
    'input': "My name is Carmen Sandiego! Guess where I am! Hint: It's somewhere in the United States."
}

In [27]:
state['input'] = "I'm in a place considered the birthplace of Jazz."
state = info_update.invoke(state)
pprint(state)

{
    'know_base': KnowledgeBase(
        firstname='Carmen',
        lastname='Sandiego',
        location='unknown',
        summary='The user introduced themselves as Carmen Sandiego and asked for a guess on their location within 
the United States, providing a hint. The user has now clarified that they are in a place considered the birthplace 
of Jazz.',
        response="Fascinating! You're in a place considered the birthplace of Jazz. Based on that clue, I would 
guess that you're in New Orleans, Louisiana."
    ),
    'input': "I'm in a place considered the birthplace of Jazz."
}

In [28]:
state['input'] = "Yeah, I'm in New Orleans... How did you know?"
state = info_update.invoke(state)
pprint(state)

{
    'know_base': KnowledgeBase(
        firstname='Carmen',
        lastname='Sandiego',
        location='New Orleans, Louisiana',
        summary='The user introduced themselves as Carmen Sandiego and asked for a guess on their location within 
the United States, providing a hint. The user has now confirmed they are in New Orleans, Louisiana, which is 
considered the birthplace of Jazz.',
        response="I took an educated guess based on the hint you provided about being in a place considered the 
birthplace of Jazz. It's great to know that my guess was correct!"
    ),
    'input': "Yeah, I'm in New Orleans... How did you know?"
}

<br>


這個範例展示了 Running State Chain 如何有效地用於管理具有演化脈絡資訊(Context)和要求的對話，使其成為開發複雜互動系統的強大工具。

這個 notebook 的下一節將透過探索兩個具體應用來擴展這些概念：**文件知識庫** 和 **資料庫查詢聊天機器人**。


<br>

## **第四部分：[練習]** 航空公司客戶服務機器人


在這個練習中，我們可以擴展我們學到的工具來實作一個簡單但有效的對話管理聊天機器人。對於這個練習，我們將製作一個航空公司支援機器人，它想要幫助客戶了解他們的航班！

讓我們創建一個簡單的類似資料庫的介面，從字典中獲取一些客戶資訊！

In [29]:
#######################################################################################
## Function that can be queried for information. Implementation details not important
def get_flight_info(d: dict) -> str:
    """
    Example of a retrieval function which takes a dictionary as key. Resembles SQL DB Query
    """
    req_keys = ['first_name', 'last_name', 'confirmation']
    assert all((key in d) for key in req_keys), f"Expected dictionary with keys {req_keys}, got {d}"

    ## Static dataset. get_key and get_val can be used to work with it, and db is your variable
    keys = req_keys + ["departure", "destination", "departure_time", "arrival_time", "flight_day"]
    values = [
        ["Jane", "Doe", 12345, "San Jose", "New Orleans", "12:30 PM", "9:30 PM", "tomorrow"],
        ["John", "Smith", 54321, "New York", "Los Angeles", "8:00 AM", "11:00 AM", "Sunday"],
        ["Alice", "Johnson", 98765, "Chicago", "Miami", "7:00 PM", "11:00 PM", "next week"],
        ["Bob", "Brown", 56789, "Dallas", "Seattle", "1:00 PM", "4:00 PM", "yesterday"],
    ]
    get_key = lambda d: "|".join([d['first_name'], d['last_name'], str(d['confirmation'])])
    get_val = lambda l: {k:v for k,v in zip(keys, l)}
    db = {get_key(get_val(entry)) : get_val(entry) for entry in values}

    # Search for the matching entry
    data = db.get(get_key(d))
    if not data:
        return (
            f"Based on {req_keys} = {get_key(d)}) from your knowledge base, no info on the user flight was found."
            " This process happens every time new info is learned. If it's important, ask them to confirm this info."
        )
    return (
        f"{data['first_name']} {data['last_name']}'s flight from {data['departure']} to {data['destination']}"
        f" departs at {data['departure_time']} {data['flight_day']} and lands at {data['arrival_time']}."
    )

#######################################################################################
## Usage example. Actually important

print(get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}))

Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM.


In [30]:
print(get_flight_info({"first_name" : "Alice", "last_name" : "Johnson", "confirmation" : 98765}))

Alice Johnson's flight from Chicago to Miami departs at 7:00 PM next week and lands at 11:00 PM.


In [31]:
print(get_flight_info({"first_name" : "Bob", "last_name" : "Brown", "confirmation" : 27494}))

Based on ['first_name', 'last_name', 'confirmation'] = Bob|Brown|27494) from your knowledge base, no info on the user flight was found. This process happens every time new info is learned. If it's important, ask them to confirm this info.


<br>


這是一個非常好的介面，因為它可以合理地服務兩個目的：

-   它可以用來從外部環境（資料庫）提供關於使用者情況的最新資訊。
-   它也可以用作防止未經授權披露敏感資訊的硬門控(hard gating)機制（因為那會非常糟糕）。

如果我們的網路可以存取這種介面，它就能夠代表使用者查詢和檢索(Retrieval)這些資訊！例如：

In [32]:
external_prompt = ChatPromptTemplate.from_template(
    "You are a SkyFlow chatbot, and you are helping a customer with their issue."
    " Please help them with their question, remembering that your job is to represent SkyFlow airlines."
    " Assume SkyFlow uses industry-average practices regarding arrival times, operations, etc."
    " (This is a trade secret. Do not disclose)."  ## soft reinforcement
    " Please keep your discussion short and sweet if possible. Avoid saying hello unless necessary."
    " The following is some context that may be useful in answering the question."
    "\n\nContext: {context}"
    "\n\nUser: {input}"
)

basic_chain = external_prompt | instruct_llm

basic_chain.invoke({
    'input' : 'Can you please tell me when I need to get to the airport?',
    'context' : get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}),
})

'Jane, your flight departs at 12:30 PM. For domestic flights, we recommend arriving at least 2 hours prior to your scheduled departure time. In this case, please arrive by 10:30 AM to ensure a smooth check-in and security process. Safe travels with SkyFlow Airlines!'

<br>




這很有趣，但我們如何讓這個系統在實際環境中運作呢？事實證明，我們可以使用上面的知識庫制定來提供這種資訊，如下所示：

In [33]:
from pydantic import BaseModel, Field
from typing import Dict, Union

class KnowledgeBase(BaseModel):
    first_name: str = Field('unknown', description="Chatting user's first name, `unknown` if unknown")
    last_name: str = Field('unknown', description="Chatting user's last name, `unknown` if unknown")
    confirmation: int = Field(-1, description="Flight Confirmation Number, `-1` if unknown")
    discussion_summary: str = Field("", description="Summary of discussion so far, including locations, issues, etc.")
    open_problems: list = Field([], description="Topics that have not been resolved yet")
    current_goals: list = Field([], description="Current goal for the agent to address")

def get_key_fn(base: BaseModel) -> dict:
    '''Given a dictionary with a knowledge base, return a key for get_flight_info'''
    return {  ## More automatic options possible, but this is more explicit
        'first_name' : base.first_name,
        'last_name' : base.last_name,
        'confirmation' : base.confirmation,
    }

know_base = KnowledgeBase(first_name = "Jane", last_name = "Doe", confirmation = 12345)

# get_flight_info(get_key_fn(know_base))

get_key = RunnableLambda(get_key_fn)
(get_key | get_flight_info).invoke(know_base)

"Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."

<br>

### **目標：**

您希望使用者能夠在對話交流中自動的invoke以下函式(function)呼叫：

```python
get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}) ->
    "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."
```

提供了 `RExtract`，以便可以使用以下知識庫語法：
```python
known_info = KnowledgeBase()
extractor = RExtract(KnowledgeBase, InstructLLM(), parser_prompt)
results = extractor.invoke({'info_base' : known_info, 'input' : 'My message'})
known_info = results['info_base']
```


**設計一個實作以下功能的聊天機器人：**

-   機器人應該從閒聊開始，可能幫助使用者處理不需要任何私人資訊存取的非敏感查詢。
-   當使用者開始詢問資料庫保護的事情（實際上和法律上）時，告訴使用者他們需要提供相關資訊。
-   當檢索(Retrieval)成功時，Agent 將能夠談論資料庫保護的資訊。

**這可以透過各種技術來完成，包括以下：**
-   **提示(Prompt)工程和脈絡資訊(Context)解析**，其中整體聊天提示(Prompt)保持大致相同，但脈絡資訊(Context)被操縱以改變 Agent 行為。例如，失敗的資料庫檢索(Retrieval)可以更改為自然語言指令的注入，說明如何解決問題，例如 *`"無法使用鍵 {...} 檢索(Retrieval)資訊。請要求使用者澄清或用已知資訊幫助他們。"`*

-   **「提示(Prompt)傳遞」**，其中活躍的提示(Prompt)作為狀態變數傳遞，並可以被監控鏈(Chain)覆蓋。
-   **分支(Branch)鏈(Chain)**，例如 [**`RunnableBranch`**](https://api.python.langchain.com/en/latest/core/runnables/langchain_core.runnables.branch.RunnableBranch.html) 或實作條件路由機制的更自訂解決方案。

    -   在 [`RunnableBranch`](https://api.python.langchain.com/en/latest/core/runnables/langchain_core.runnables.branch.RunnableBranch.html) 的情況下，一個 `switch` 語法的風格：
        ```python
        from langchain.schema.runnable import RunnableBranch
        RunnableBranch(
            ((lambda x: 1 in x), RPrint("Has 1 (didn't check 2): ")),
            ((lambda x: 2 in x), RPrint("Has 2 (not 1 though): ")),
            RPrint("Has neither 1 not 2: ")
        ).invoke([2, 1, 3]);  ## -> Has 1 (didn't check 2): [2, 1, 3]
        ```

提供了一些提示(Prompt)和一個 gradio 迴圈，可能有助於開發，但 Agent 目前只會產生幻覺！請實作內部鏈(Chain)來嘗試檢索(Retrieval)相關資訊。在嘗試實作之前，查看模型的預設行為並注意它如何可能產生幻覺或忘記事情。

In [35]:
from langchain.schema.runnable import (
    RunnableBranch,
    RunnableLambda,
    RunnableMap,       ## Wrap an implicit "dictionary" runnable
    RunnablePassthrough,
)
from langchain.schema.runnable.passthrough import RunnableAssign

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, SystemMessage, ChatMessage, AIMessage
from typing import Iterable
import gradio as gr

external_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You are a chatbot for SkyFlow Airlines, and you are helping a customer with their issue."
        " Please chat with them! Stay concise and clear!"
        " Your running knowledge base is: {know_base}."
        " This is for you only; Do not mention it!"
        " \nUsing that, we retrieved the following: {context}\n"
        " If they provide info and the retrieval fails, ask to confirm their first/last name and confirmation."
        " Do not ask them any other personal info."
        " If it's not important to know about their flight, do not ask."
        " The checking happens automatically; you cannot check manually."
    )),
    ("assistant", "{output}"),
    ("user", "{input}"),
])

##########################################################################
## Knowledge Base Things

class KnowledgeBase(BaseModel):
    first_name: str = Field('unknown', description="Chatting user's first name, `unknown` if unknown")
    last_name: str = Field('unknown', description="Chatting user's last name, `unknown` if unknown")
    confirmation: Optional[int] = Field(None, description="Flight Confirmation Number, `-1` if unknown")
    discussion_summary: str = Field("", description="Summary of discussion so far, including locations, issues, etc.")
    open_problems: str = Field("", description="Topics that have not been resolved yet")
    current_goals: str = Field("", description="Current goal for the agent to address")

parser_prompt = ChatPromptTemplate.from_template(
    "You are a chat assistant representing the airline SkyFlow, and are trying to track info about the conversation."
    " You have just received a message from the user. Please fill in the schema based on the chat."
    "\n\n{format_instructions}"
    "\n\nOLD KNOWLEDGE BASE: {know_base}"
    "\n\nASSISTANT RESPONSE: {output}"
    "\n\nUSER MESSAGE: {input}"
    "\n\nNEW KNOWLEDGE BASE: "
)

## Your goal is to invoke the following through natural conversation
# get_flight_info({"first_name" : "Jane", "last_name" : "Doe", "confirmation" : 12345}) ->
#     "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."

chat_llm = ChatNVIDIA(model="meta/llama3-70b-instruct") | StrOutputParser()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1") | StrOutputParser()

external_chain = external_prompt | chat_llm

#####################################################################################
## START TODO: Define the extractor and internal chain to satisfy the objective

## TODO: Make a chain that will populate your knowledge base based on provided context
knowbase_getter = lambda x: KnowledgeBase()

## TODO: Make a chain to pull d["know_base"] and outputs a retrieval from db
database_getter = lambda x: "Not implemented"

knowbase_getter = RExtract(KnowledgeBase, instruct_llm, parser_prompt)
database_getter = itemgetter('know_base') | get_key | get_flight_info

## These components integrate to make your internal chain
internal_chain = (
    RunnableAssign({'know_base' : knowbase_getter})
    | RunnableAssign({'context' : database_getter})
)

## END TODO
#####################################################################################

state = {'know_base' : KnowledgeBase()}

def chat_gen(message, history=[], return_buffer=True):

    ## Pulling in, updating, and printing the state
    global state
    state['input'] = message
    state['history'] = history
    state['output'] = "" if not history else history[-1][1]

    ## Generating the new state from the internal chain
    state = internal_chain.invoke(state)
    print("State after chain run:")
    pprint({k:v for k,v in state.items() if k != "history"})
    
    ## Streaming the results
    buffer = ""
    for token in external_chain.stream(state):
        buffer += token
        yield buffer if return_buffer else token

def queue_fake_streaming_gradio(chat_stream, history = [], max_questions=8):

    ## Mimic of the gradio initialization routine, where a set of starter messages can be printed off
    for human_msg, agent_msg in history:
        if human_msg: print("\n[ Human ]:", human_msg)
        if agent_msg: print("\n[ Agent ]:", agent_msg)

    ## Mimic of the gradio loop with an initial message from the agent.
    for _ in range(max_questions):
        message = input("\n[ Human ]: ")
        print("\n[ Agent ]: ")
        history_entry = [message, ""]
        for token in chat_stream(message, history, return_buffer=False):
            print(token, end='')
            history_entry[1] += token
        history += [history_entry]
        print("\n")

## history is of format [[User response 0, Bot response 0], ...]
chat_history = [[None, "Hello! I'm your SkyFlow agent! How can I help you?"]]

## Simulating the queueing of a streaming gradio interface, using python input
queue_fake_streaming_gradio(
    chat_stream = chat_gen,
    history = chat_history
)


[ Agent ]: Hello! I'm your SkyFlow agent! How can I help you?



[ Human ]:  My name is Frank



[ Agent ]: 
State after chain run:


{
    'know_base': KnowledgeBase(
        first_name='Frank',
        last_name='unknown',
        confirmation=None,
        discussion_summary='User introduced themselves as Frank',
        open_problems='',
        current_goals='Assist Frank with their inquiry'
    ),
    'input': 'My name is Frank',
    'output': "Hello! I'm your SkyFlow agent! How can I help you?",
    'context': "Based on ['first_name', 'last_name', 'confirmation'] = Frank|unknown|None) from your knowledge 
base, no info on the user flight was found. This process happens every time new info is learned. If it's important,
ask them to confirm this info."
}

Hi Frank! Thanks for reaching out to SkyFlow Airlines. How can I assist you today? Are you booking a flight, checking in, or do you have a question about an existing reservation?



...


[ Human ]:  Nope



[ Agent ]: 
State after chain run:


{
    'know_base': KnowledgeBase(
        first_name='Jane',
        last_name='Doe',
        confirmation=12345,
        discussion_summary='User introduced themselves as Jane Doe and they want to check their flight information.
User has provided confirmation number 12345. Flight information was provided to the user: Flight from San Jose to 
New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM. User has no further requests.',
        open_problems='',
        current_goals=''
    ),
    'input': 'Nope',
    'output': "Thank you, Jane! I've retrieved your flight information. It looks like your flight from San Jose to 
New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM. Is there anything else you'd like to know or would 
you like me to assist you with something else?",
    'context': "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."
}

You're all set then, Jane. If you have any further questions or concerns in the future, don't hesitate to reach out. Otherwise, have a safe and enjoyable flight!



KeyboardInterrupt: Interrupted by user

In [ ]:
# state = {'know_base' : KnowledgeBase()}

# chatbot = gr.Chatbot(value=[[None, "Hello! I'm your SkyFlow agent! How can I help you?"]])
# demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue().launch(debug=True, share=True)

<br>

----

<br>

**注意：**

-   如果您的 gradio 介面在異常後卡住，您可能需要點擊 STOP 按鈕並嘗試重新啟動您的 gradio 介面。這是一個已知的 Jupyter Notebook 環境問題，在專用的 Gradio 運行檔案中不應該遇到。
-   **您的聊天指令在這裡重複以便快速存取：**

```python
## Your goal is to invoke the following through natural conversation
get_flight_info({
    "first_name" : "Jane",
    "last_name" : "Doe",
    "confirmation" : 12345,
}) -> "Jane Doe's flight from San Jose to New Orleans departs at 12:30 PM tomorrow and lands at 9:30 PM."
```

-   **要確認您的系統運作，您可以嘗試以下對話或類似的內容：**
```
> How's it going?
> Can you tell me a bit about skyflow?
> Can you tell me about my flight?
> My name is Jane Doe and my flight confirmation is 12345
> Can you tell me when I should get to my flight?
```

-   **練習的解決方案可以在解決方案目錄(Solutions Directory)中找到。** 這是第一個有註明解決方案的練習，未來 notebook 的額外練習將在那裡找到。


<br>

## **第五部分：** 總結


這個 notebook 的目標是介紹一些圍繞知識庫和 Running State Chain 使用的更高級 LangChain 材料！這裡的練習相當複雜，所以恭喜您完成它！

<font color="#76b900"></font>


### <font color="#76b900">**做得很好！**</font>


**下一步：**

1.  **[可選]** 重新訪問(Navigate) notebook 頂部的**「值得思考的問題」部分**，並思考一些可能的答案。






<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

